In [1]:
import random
import torch
import numpy as np
from torch import nn,optim
from model_debug import RN
from functions import from_batch
from torch.autograd import Variable

In [2]:
# define hyperparameters
batch_size = 64
embed_size = 64
en_hidden_size = 32
mlp_hidden_size = 256
epochs = 30

# optional: if you're starting off from a previous model
startoff=0

In [3]:
# load dataset
with open('train_10k.txt') as f:
    lines=f.readlines()
if len(lines)%batch_size==0:
    num_batches = int(len(lines)/batch_size)
else:
    num_batches = int(len(lines)/batch_size)

# load vocabulary
word2idx = np.load('word2idx.npy').item()
idx2word = np.load('idx2word.npy').item()

In [4]:
vocab_size = len(word2idx)

In [5]:
if startoff>0:
    rn = torch.load('saved/rn_%d.pth' %startoff)
else:
    rn = RN(vocab_size, embed_size, en_hidden_size, mlp_hidden_size)
if torch.cuda.is_available():
    rn = rn.cuda()
opt = optim.Adam(rn.parameters(),lr=2e-4)
criterion = nn.CrossEntropyLoss()

In [6]:
# for validation
def validate(rn, val_set):
    S, Q, A = from_batch(val_set)
    out = rn(S, Q)
    O = torch.max(out,1)[1].cpu().data.numpy().squeeze()
    score = np.array(O==A,int)
    total = len(score)
    correct = sum(score)
    return total,correct

with open('test_10k.txt') as f:
    val_set=f.readlines()[:batch_size]

In [7]:
# training
# for epoch in range(epochs):
for epoch in range(10):
    random.shuffle(lines) # shuffle lines
    for i in range(num_batches):
#     for i in range(5):
        opt.zero_grad()
#         print(i)
        batch = lines[i*batch_size:(i+1)*batch_size]
        S,Q,A = from_batch(batch)
        out = rn(S,Q)
        A = Variable(torch.LongTensor(A))
        if torch.cuda.is_available():
            A = A.cuda()
        loss = criterion(out,A)
        loss.backward()
        opt.step()
        if i % 50==0:
            print("loss for %d/%d: %1.3f" % (i,num_batches,loss.data[0]))
    print("loss for epoch %d: %1.3f" % (epoch,loss.data[0]))
    model_name = 'rn_%d.pth' % (epoch+1+startoff)
    if i % 5==0:
        torch.save(obj=rn,f='saved/'+model_name)
    total, correct = validate(rn, val_set)
    print("Validation score: ",correct*1.0/total)

loss for 0/3125: 5.711
loss for 50/3125: 4.233
loss for 100/3125: 2.874
loss for 150/3125: 3.000
loss for 200/3125: 2.932
loss for 250/3125: 3.333
loss for 300/3125: 3.020
loss for 350/3125: 2.556
loss for 400/3125: 3.063
loss for 450/3125: 3.170
loss for 500/3125: 2.779
loss for 550/3125: 3.032
loss for 600/3125: 3.468
loss for 650/3125: 2.632
loss for 700/3125: 2.778
loss for 750/3125: 2.968
loss for 800/3125: 2.068
loss for 850/3125: 2.882
loss for 900/3125: 2.318
loss for 950/3125: 2.641
loss for 1000/3125: 2.886
loss for 1050/3125: 3.174
loss for 1100/3125: 2.896
loss for 1150/3125: 3.014
loss for 1200/3125: 2.641
loss for 1250/3125: 2.799
loss for 1300/3125: 2.565
loss for 1350/3125: 2.971
loss for 1400/3125: 2.497
loss for 1450/3125: 2.605
loss for 1500/3125: 2.826
loss for 1550/3125: 3.295
loss for 1600/3125: 2.918
loss for 1650/3125: 2.729
loss for 1700/3125: 2.937
loss for 1750/3125: 2.359
loss for 1800/3125: 2.824
loss for 1850/3125: 3.035
loss for 1900/3125: 2.492
loss for 

KeyboardInterrupt: 

In [11]:
# validation
# load dataset
with open('test_10k.txt') as f:
    lines=f.readlines()
num_batches = int(len(lines)/batch_size)

# load vocabulary
word2idx = np.load('word2idx.npy').item()
idx2word = np.load('idx2word.npy').item()

rn = torch.load('saved/rn_4.pth')
if torch.cuda.is_available():
    rn = rn.cuda()

# for epoch in range(epochs):
total = 0
correct = 0
for epoch in range(1):
    random.shuffle(lines) # shuffle lines
    for i in range(num_batches):
#     for i in range(5):
        opt.zero_grad()
#         print(i)
        batch = lines[i*batch_size:(i+1)*batch_size]
        S,Q,A = from_batch(batch)
        out = rn(S,Q)
        # out : [batch x vocab_size]
        O = torch.max(out,1)[1].cpu().data.numpy().squeeze()
        score = np.array(O==A, int)
        total+=len(score)
        correct+=sum(score)

In [20]:
score = np.array(O.squeeze()==A, int)

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [ ]:
sum(S[2])

In [ ]:
q

In [ ]:
a